# <B> Setup for Anormaly Detection with SageMaker </B>
* Container: codna_python3

## AutoReload

In [1]:
%load_ext autoreload
%autoreload 2

## 0. Install packages

In [2]:
install_needed = True  # should only be True once
# install_needed = False

In [3]:
%%bash
#!/bin/bash

DAEMON_PATH="/etc/docker"
MEMORY_SIZE=10G

FLAG=$(cat $DAEMON_PATH/daemon.json | jq 'has("data-root")')
# echo $FLAG

if [ "$FLAG" == true ]; then
    echo "Already revised"
else
    echo "Add data-root and default-shm-size=$MEMORY_SIZE"
    sudo cp $DAEMON_PATH/daemon.json $DAEMON_PATH/daemon.json.bak
    sudo cat $DAEMON_PATH/daemon.json.bak | jq '. += {"data-root":"/home/ec2-user/SageMaker/.container/docker","default-shm-size":"'$MEMORY_SIZE'"}' | sudo tee $DAEMON_PATH/daemon.json > /dev/null
    sudo service docker restart
    echo "Docker Restart"
fi

Already revised


In [4]:
import sys
import IPython

if install_needed:
    print("installing deps and restarting kernel")
    !{sys.executable} -m pip install -U pip
    !{sys.executable} -m pip install -U awswrangler #==2.20.1
    !{sys.executable} -m pip install -U sagemaker
    !{sys.executable} -m pip install -U datasets transformers s3fs
    !sudo curl -L "https://github.com/docker/compose/releases/download/v2.26.1/docker-compose-$(uname -s)-$(uname -m)" -o /usr/local/bin/docker-compose
    !sudo chmod +x /usr/local/bin/docker-compose
        
    IPython.Application.instance().kernel.do_shutdown(True)

installing deps and restarting kernel
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 122.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 MB 68.6 MB/s eta 0:00:00:00:01
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 19.0.0
    Uninstalling pyarrow-19.0.0:
      Successfully uninstalled pyarrow-19.0.0
  Attempting uninstall: botocore
    Found existing installation: botocore 1.36.3
    Uninstalling botocore-1.36.3:
      Successfully uninstalled botocore-1.36.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.19.0 requires botocore<1.36.4,>=1.36.0, but you have botocore 1.36.21 which is incompatible.
awscli 1.37.18 requires botocore==1.36.18, but you have botocore 1.36.21 which is incompatible.
  Attempting uninstall: dill
    Found existing installation: dill 0.3.8
    Uninstalling dill-0.

## 1. Set roles

In [2]:
from sagemaker import get_execution_role

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[02/15/25 23:22:53] INFO     Found credentials from IAM Role:                                   ]8;id=136430;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=690833;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [3]:
strSageMakerRoleName = get_execution_role().rsplit('/', 1)[-1]
print (f"SageMaker Execution Role Name: {strSageMakerRoleName}")

                    INFO     Found credentials from IAM Role:                                   ]8;id=597376;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=521279;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

SageMaker Execution Role Name: AmazonSageMaker-ExecutionRole-20221206T163436


## 1.1 Attach IAM polich to sagemaker execution role (<b>with console</b>)
> **S3**: "arn:aws:iam::aws:policy/AmazonS3FullAccess"

### 1.1 Attach IAM polich to sagemaker execution role (<b>with console</b>)
> step 1. IAM console 로 이동    
>
> step 2. 왼쪽 네비게이터에서  "Role" 선택  
> ![nn](imgs/Role.png)  
>
> step 3. SageMaker Execution Role 검색 후 role 선택 (상위 cell output 참조)  
> ![nn](imgs/search-by-rolename.png)  
>
> step 4. "attach policies" 메뉴 선택 
> ![nn](imgs/attach-policy-menu.png)  
>
> step 5. "IAMFullAccess" policy 검색 후 attach 
> ![nn](imgs/attach-policy.png) 

In [4]:
from utils.iam import iam_handler

In [5]:
iam = iam_handler()

[02/15/25 23:22:55] INFO     Found credentials from IAM Role:                                   ]8;id=225763;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=55203;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

### 1.2 Attach policy - sagemaker execution role
> SSM  
> S3 <BR>
> Kinesis <BR>
> KinesisAnalytics <BR>

In [6]:
listPolicyArn=[
    "arn:aws:iam::aws:policy/AmazonSSMFullAccess",
    "arn:aws:iam::aws:policy/AmazonS3FullAccess",
    #"arn:aws:iam::aws:policy/AmazonKinesisFullAccess",
    #"arn:aws:iam::aws:policy/AmazonKinesisAnalyticsFullAccess",
]
for strPolicyArn in listPolicyArn: iam.attach_policy(strSageMakerRoleName, strPolicyArn)

## 2. Set default parameters

In [7]:
import boto3
import sagemaker

### Bucket / Prefix 설정

In [8]:
strRegionName = boto3.Session().region_name
strAccountId = boto3.client("sts").get_caller_identity().get("Account")
strBucketName = 'sm-anomaly-detection-dongjin' # <-- 사용할 bucket 명을 추가해 주세요. ex) sagemaker-us-east-1-123456789123, sm-nemo-bucket
strPrefix = 'ad-ts' ## <-- 작업할 prefix 명을 추가해 주세요. ex) nemo-test, nemo-asr

## 3. Extract & Upload dataset

In [9]:
import os
from utils.s3 import s3_handler

In [10]:
strLocalDataPath = "./data"
!tar -zxvf $strLocalDataPath/data.tar.gz --directory=data

./
./fault_label_10T.csv
./clicks.csv
./clicks_10T.csv
./fault_label_1T.csv
./clicks_1T.csv


* create bucket

In [13]:
s3 = s3_handler('us-west-2')

This is a S3 handler with [us-west-2] region.


In [15]:
s3.create_bucket(strBucketName)

CREATE:[sm-anomaly-detection-dongjin] Bucket was created successfully


True

* updoad data to s3

In [16]:
source_dir, target_bucket, target_dir = strLocalDataPath, strBucketName, "data"
s3.upload_dir(source_dir, target_bucket, target_dir)

Upload:[./data] was uploaded to [s3://sm-anomaly-detection-dongjin/data]successfully


## 4. [Optional] AWS Systems Manager Parameter Store 를 이용한 파라미터 저장/활용
- [AWS Systems Manager Parameter Store](https://docs.aws.amazon.com/systems-manager/latest/userguide/systems-manager-parameter-store.html)
- Attach IAM polich to sagemaker execution role (<b>with console</b>)
> **SSM**: "arn:aws:iam::aws:policy/AmazonSSMFullAccess"<BR>

In [17]:
from utils.ssm import parameter_store

In [18]:
pm = parameter_store(strRegionName)

In [19]:
pm.put_params(key="PREFIX", value=strPrefix, overwrite=True)
pm.put_params(key="-".join([strPrefix, "REGION"]), value=strRegionName, overwrite=True)
pm.put_params(key="-".join([strPrefix, "BUCKET"]), value=strBucketName, overwrite=True)
pm.put_params(key="-".join([strPrefix, "SAGEMAKER-ROLE-ARN"]), value=get_execution_role(), overwrite=True)
pm.put_params(key="-".join([strPrefix, "ACCOUNT-ID"]), value=strAccountId, overwrite=True)
pm.put_params(key="-".join([strPrefix, "S3-DATA-PATH"]), value=f"s3://{strBucketName}/data", overwrite=True)

'Store suceess'

In [20]:
print (f'PREFIX: {pm.get_params(key="PREFIX")}')
print (f'REGION: {pm.get_params(key="-".join([strPrefix, "REGION"]))}')
print (f'BUCKET: {pm.get_params(key="-".join([strPrefix, "BUCKET"]))}')
print (f'SAGEMAKER-ROLE-ARN: {pm.get_params(key="-".join([strPrefix, "SAGEMAKER-ROLE-ARN"]))}')
print (f'ACCOUNT-ID: {pm.get_params(key="-".join([strPrefix, "ACCOUNT-ID"]))}')
print (f'S3-DATA-PATH: {pm.get_params(key="-".join([strPrefix, "S3-DATA-PATH"]))}')

PREFIX: ad-ts
REGION: us-west-2
BUCKET: sm-anomaly-detection-dongjin
SAGEMAKER-ROLE-ARN: arn:aws:iam::419974056037:role/service-role/AmazonSageMaker-ExecutionRole-20221206T163436
ACCOUNT-ID: 419974056037
S3-DATA-PATH: s3://sm-anomaly-detection-dongjin/data
